# Dataset processor

This notebook does the following:
1. Compile Faust DSP files to CPP files
2. Compile said CPP files with clang++
3. Processes every file from the SOL Dataset with every effect

### Dependencies:
First, you can install `clang++` and `faust` from the repos:
```bash
sudo apt-get install clang faust
```

Then, install [libsndfile](https://github.com/erikd/libsndfile/):
```bash
git clone git://github.com/erikd/libsndfile.git
sudo apt install autoconf autogen automake build-essential libasound2-dev \
    libflac-dev libogg-dev libtool libvorbis-dev pkg-config python
./autogen.sh
./configure --enable-werror
make
make install
```


In [2]:
import torch
from torch.utils import data
import glob
import os

## Dataset transformation

In [18]:
types = ('*.pdf', '*.cpp') # the tuple of file types
files_grabbed = []
for files in types:
    files_grabbed.extend(glob.glob(files))

#for filename in glob.iglob('SOL-ordinario/**/*.wav', recursive=True):
#    print(filename)	
files_grabbed.extend(glob.iglob('SOL-ordinario/**/*.wav', recursive=True))

print(files_grabbed)
	

['SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#3-f.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#3-ff.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#3-mf.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#3-pp.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#4-f.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#4-ff.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#4-mf.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#4-pp.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#5-f.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#5-ff.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#5-mf.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#5-pp.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A3-f.wav', 'SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A3-ff.wav', 'SOL-ordinario/Brass/English-Horn/ordin

## Definitions

In [30]:
class Serge(data.Dataset):
    def __init__(self, files, process):
        types = ('wav', 'aif', 'aiff')
        self.liste = []
        for extension in types:
            files_ = files + "." + extension
            
            self.liste.extend(glob.iglob(files_, recursive=True))      
        print("Preprocessing stuff... ", end="")
        if process:
            for elm in self.liste:
                [x,fs] = li.load(elm, sr=22050)
                x = self.pad(x,34560)
                assert(len(x)==34560)
                mel = li.filters.mel(fs,2048,500)
                S = torch.from_numpy(mel.dot(abs(li.stft(x,n_fft=2048,win_length=2048, hop_length=256, center=False)))).float()
                S = S/torch.max(S)
                torch.save(S,elm.replace(".wav",".pt"))
        
        print("Done!")
    def __getitem__(self,i):
        return torch.load(self.liste[i].replace(".wav",".pt"))
    
    def __len__(self):
        return len(self.liste)
    
    def pad(self,x,n):
        m = len(x)
        if m<n:
            return np.concatenate([x,np.zeros(n-m)])
        else:
            return x[:n]

			
			
serge = Serge(files='SOL-ordinario/**/*', process=False)
sergeLoader = data.DataLoader(serge, batch_size=8, shuffle=True, drop_last=True)
print('Dataset size: {}'.format(len(serge.liste)))

Preprocessing stuff... Done!
Dataset size: 2020


In [35]:
print(serge.liste[0])

SOL-ordinario/Brass/English-Horn/ordinario/EH_nA-4_120-A#3-f.wav


In [37]:
test_file_path = serge.liste[0]
for file in glob.iglob('csound/*.csd'):
    print(file)

#bashCommand = "for i in *.aif; do csound -d -A -n --omacro:theFile=$i --omacro:outFile=${i%.*}.L.aif --omacro:sig=asig1 batchfx2.csd ;  done | for i in *.aif; do csound -d -A -n --omacro:theFile=$i --omacro:outFile=${i%.*}.R.aif --omacro:sig=asig2 batchfx2.csd ;  done "
#subprocess.Popen(bashCommand)

csound/distortion.csd


## DSP files compilation
1. compile faust to cpp: `faust -a sndfile.cpp gain-stereo.dsp > gain-stereo.cpp`
2. compile cpp to executable: `clang++ -o gain-stereo lsndfile gain-stereo.cpp`

In [13]:
REGENERATE_CPP = False
COMPILE_CPP = True

if COMPILE_CPP:
    os.system('mkdir -p bin')

for file in glob.glob('faust/**/*.dsp',recursive=True):
    filename_wout_extension = file[len('faust/'):-len('.dsp')]
    cpp_filename = "faust_cpp/"+filename_wout_extension + ".cpp"
    bin_path = "bin/" + filename_wout_extension
    if REGENERATE_CPP:
        bashCommand = "faust -a sndfile.cpp "+ file + " > " + cpp_filename
        print(bashCommand)
        os.system(bashCommand)
    if COMPILE_CPP:
        bashCommand = "clang++ -o "+ bin_path + " -lsndfile " + cpp_filename
        print(bashCommand)
        os.system(bashCommand)
print('DSP files compilation done!')

clang++ -o bin/autovocoder1 -lsndfile faust_cpp/autovocoder1.cpp
clang++ -o bin/autovocoder2 -lsndfile faust_cpp/autovocoder2.cpp
clang++ -o bin/autowah1 -lsndfile faust_cpp/autowah1.cpp
clang++ -o bin/autowah2 -lsndfile faust_cpp/autowah2.cpp
clang++ -o bin/autowah3 -lsndfile faust_cpp/autowah3.cpp
clang++ -o bin/autowah4 -lsndfile faust_cpp/autowah4.cpp
clang++ -o bin/flanger_demo -lsndfile faust_cpp/flanger_demo.cpp
clang++ -o bin/freeverb_demo -lsndfile faust_cpp/freeverb_demo.cpp
clang++ -o bin/phaser2_demo -lsndfile faust_cpp/phaser2_demo.cpp
clang++ -o bin/wah1 -lsndfile faust_cpp/wah1.cpp
clang++ -o bin/wah2 -lsndfile faust_cpp/wah2.cpp
clang++ -o bin/wah3 -lsndfile faust_cpp/wah3.cpp
clang++ -o bin/wah4 -lsndfile faust_cpp/wah4.cpp
clang++ -o bin/distortion/cubic/cubic_0.25_0.5 -lsndfile faust_cpp/distortion/cubic/cubic_0.25_0.5.cpp
clang++ -o bin/distortion/cubic/cubic_0.25_0 -lsndfile faust_cpp/distortion/cubic/cubic_0.25_0.cpp
clang++ -o bin/distortion/cubic/cubic_0.5_0.5 -